Get original DataFrame

In [ ]:
import sys
import plotly.graph_objects as go
import pandas as pd

import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
sys.path.insert(0, os.path.abspath('../..'))

from python_scripts.api_calls import fetch_item_to_df, fetch_items, get_cookie_from_blob


dailyCookie = get_cookie_from_blob()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
#print(items[4])

#print(current_item.tail())
#print(non_aggregated_item.tail())

df = current_item

os. getcwd()

convert an array of values into a dataset matrix

In [ ]:

def create_dataset(dataset, look_back=1):
 dataX, dataY = [], []
 print(len(dataset))
 for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
 return np.array(dataX), np.array(dataY)


#printx,printy = create_dataset(dataset, 1)
#print(printx)


Loads and normalises dataset

In [ ]:
processed_df = df["price_usd"]
tf.random.set_seed(0)



dataset = processed_df.values
dataset = dataset.astype('float32')
print(dataset)
#print(processed_df)
#normalize the dataset
dataset = dataset.reshape(-1, 1)
print(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print(dataset)

split into training and testing

In [ ]:

train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

print(len(train), len(test))



# train_size = int(len(dataset) * 0.67)

# X_train, X_test = X[:split_point], X[split_point:]
# y_train, y_test = y[:split_point], y[split_point:]

Reshapes into X=t and Y=t+1

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
#print(len(train))
#print(len(trainX))
#print(len(trainY))
counter = 0
for i in range(len(train)-look_back-1):
        counter+=1
        
print (counter)

Reshapes input for samples, timesteps and features

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Creates and fit the LSTM network

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

Makes predictions, inverts predictions, and calculate root mean sqaured error

In [ ]:
#trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
unscaledtrainY = scaler.inverse_transform([trainY])
unscaledtestPredict = scaler.inverse_transform(testPredict)
unscaledtestY = scaler.inverse_transform([testY])

# calculate root mean squared error
#trainScore = np.sqrt(mean_squared_error(scaledtrainY[0], trainPredict[:,0]))
#print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(unscaledtestY[0], unscaledtestPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
print(unscaledtestY, unscaledtestPredict)

Shifts predicitons and plots graphs

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)

trainPredictPlot[:, :] = np.nan
print(len(unscaledtestPredict))
print(len(unscaledtestY))
#trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
#testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
# plt.plot(scaler.inverse_transform(dataset))
# plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
# test